<a href="https://colab.research.google.com/github/sheldon-o/fare-prediction-ds-project/blob/main/fare_prediction_ds_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

#reading file 


In [27]:
#train_data = pd.read_excel("E:\data science\Data_Train.xlsx")
train_data = pd.read_excel('/Untitled Folder/Data_Train.xlsx')


In [ ]:
train_data.head()

In [ ]:
train_data.shape

task deal with missing values
task data cleaning to make data ready for data anallysis


In [ ]:
train_data.isna().sum()

In [19]:
train_data.dropna(inplace=True)

In [ ]:
train_data.isna().sum()

In [ ]:
train_data.dtypes

panda assign data into object so we have to convert them ,,data ,time :data of journey,arrivel time,departure time

In [29]:
def change_into_datetime(col):
  train_data[col] = pd.to_datetime(train_data[col])


In [ ]:
train_data.columns

In [31]:
for i in ['Date_of_Journey', 'Dep_Time', 'Arrival_Time' ]:
  change_into_datetime(i)

In [ ]:
train_data.dtypes

to make sure that our program read date and month we seperate them 

In [34]:
train_data['journey_day'] = train_data['Date_of_Journey'].dt.day
train_data['journey_month'] = train_data['Date_of_Journey'].dt.month


In [ ]:
train_data.head()

In [ ]:
train_data.drop('Date_of_Journey',axis = 1 , inplace = True)

In [ ]:
train_data.head()